0.4.1


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# Import comet_ml in the top of your file
from comet_ml import Experiment
##Needs to be imported before sklearn
from sklearn.model_selection import train_test_split
import sklearn

import torch
import torch.utils.data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn, optim, sigmoid
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.nn import modules

from torch.autograd import Variable

#from torchaudio import transforms
#from torchaudio import Datasets

import os
import sys
import shutil
from glob import glob
import datetime
import re
from tqdm import tqdm
import time


#from torchviz import make_dot, make_dot_from_trace


In [2]:
# Check device type


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device


device(type='cpu')

In [4]:
def data_creator(T = 20, L = 10000, N = 1000):
    np.random.seed(2)



    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    data = sklearn.preprocessing.normalize(data)

    torch.save(data, open('traindata.pt', 'wb'))

In [5]:
# input to LSTM is sequence length, batch, input_size   
#sequence length ---- how many time steps RNN
#batch 


In [46]:
class RVAE_model(nn.Module):
    def __init__(self):
        super(RVAE_model, self).__init__()

        self.rnn1 = nn.GRU(10000, 400, 2)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.rnn2 = nn.GRU(400, 10000, 2)

    def encode(self, x):
        h1 = F.relu(self.rnn1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return sigmoid(self.rnn3(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar



In [66]:
class VAE_model(nn.Module):
    def __init__(self):
        super(VAE_model, self).__init__()

        self.fc1 = nn.Linear(1000, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 1000)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [67]:
class sineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = torch.load('traindata.pt')
        self.label = []
        vals = len(self.data)
        for val in range(vals):
            self.label.append(val)

        self.data, self.label = map(torch.FloatTensor, (self.data, self.label))
        
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
        

In [37]:

def get_model():
    
    model = RVAE_model().to(device)

    
    return model, optim.Adam(model.parameters(), lr = lr)


In [69]:
def get_data(train_dataset, test_dataset, bs):
    return (DataLoader(train_dataset, batch_size = batch_size),
    DataLoader(test_dataset, batch_size = batch_size * 2))

In [70]:
###Need data_creator function

data_creator()

In [71]:
hidden_dim = 10000
latent = 5
input_dim = 5
epochs = 10000
seq_len = 10
batch_size = 32
input_size = 997

features = 997
latent =50


lr = 1e-6

In [72]:
 
full_dataset = sineDataset( )
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dl, test_dl = get_data(train_dataset, test_dataset, batch_size)


In [73]:
def loss_function(recon_batch, mu, logvar, x):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    MSE = F.binary_cross_entropy(recon_batch, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return MSE + KLD

In [74]:
def loss_batch(model, loss_function, xb, opt=None):
    recon_batch, mu,logvar = model(xb)
    loss = loss_function(recon_batch, mu,logvar, xb)
   
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [84]:
def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    for epoch in range(epochs):
        model.train()
        
        for xb in train_dl:
            loss_batch(model, loss_func, xb, opt)
            model.eval()
            with torch.no_grad():
                losses,nums = zip(*[loss_batch(model, loss_func, xb)
                                    for xb in test_dl])
            val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
            print(epoch, val_loss)

## Full training in 3 lines of code

In [85]:

model, opt = get_model()
#x = torch.randn(1, 10000)
#make_dot(model(x), params=dict(model.named_parameters()))

       
    

In [86]:
fit(epochs, model, loss_function, opt, train_dl, test_dl)

RuntimeError: input must have 3 dimensions, got 2